In [1]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master('local')
         .appName('merge-schema-driver')
         # Add AVRO package
         .config("spark.jars.packages", "org.apache.spark:spark-avro_2.11:2.4.5")
         .getOrCreate())
sc = spark.sparkContext

# Set dynamic partitions to overwrite only the partition in DF
spark.conf.set('spark.sql.sources.partitionOverwriteMode', 'dynamic')

# Generate test data

In [2]:
import datetime

# Set up parameters
file_format = "parquet"
entity = "test_data_{}".format(file_format)
data_path = "/home/jovyan/work/spark-data/raw/{}".format(entity)
num_rows = 10

%run ./modules/01_create_simple_schema.ipynb 
gen_data_simple_schema(data_path, datetime.date(2020,1,1), num_rows, file_format)

%run ./modules/02_add_nested_structure_to_schema.ipynb
gen_data_add_nested_struct(data_path, datetime.date(2020,2,1), num_rows, file_format)

%run ./modules/03_add_column_to_schema.ipynb
gen_data_add_columns(data_path, datetime.date(2020,3,1), num_rows, file_format)

%run ./modules/04_change_datatype_add_struct.ipynb
gen_data_change_datatype_add_struct(data_path, datetime.date(2020,4,1), num_rows, file_format)

%run ./modules/05_change_column_name.ipynb
gen_data_change_column_name(data_path, datetime.date(2020,5,1), num_rows, file_format)

%run ./modules/06_remove_column.ipynb
gen_data_remove_column(data_path, datetime.date(2020,6,1), num_rows, file_format)


Partition created: /home/jovyan/work/spark-data/raw/test_data_parquet/date=2020-01-01
# Rows: 10
Schema:
root
 |-- identifier: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- date: date (nullable = true)



Partition created: /home/jovyan/work/spark-data/raw/test_data_parquet/date=2020-02-01
# Rows: 10
Schema:
root
 |-- identifier: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- address: struct (nullable = true)
 |    |-- address: string (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- state: string (nullable = true)
 |    |-- postal_code: string (nullable = true)
 |-- date: date (nullable = true)



Partition created: /home/jovyan/work/spark

# Run Merge Schema

In [3]:
# Load functions
%run ./modules/mergeSchema-functions.ipynb
%run ./modules/helpers.ipynb

# Parameters set up
ctrl_file = "/home/jovyan/work/spark-data/raw/last_read_control/{}.json".format(entity)

# Read partitions and merge schemas
rdd_json = merge_schemas(data_path, file_format, "I", ctrl_file)
if rdd_json == None:
    print("No new files to process")
else:
    df = spark.read.json(rdd_json)

Different schemas identified:
{
    "0": {
        "init_path": "/home/jovyan/work/spark-data/raw/test_data_parquet/date=2020-01-01",
        "final_path": "/home/jovyan/work/spark-data/raw/test_data_parquet/date=2020-01-01"
    },
    "1": {
        "init_path": "/home/jovyan/work/spark-data/raw/test_data_parquet/date=2020-02-01",
        "final_path": "/home/jovyan/work/spark-data/raw/test_data_parquet/date=2020-02-01"
    },
    "2": {
        "init_path": "/home/jovyan/work/spark-data/raw/test_data_parquet/date=2020-03-01",
        "final_path": "/home/jovyan/work/spark-data/raw/test_data_parquet/date=2020-03-01"
    },
    "3": {
        "init_path": "/home/jovyan/work/spark-data/raw/test_data_parquet/date=2020-04-01",
        "final_path": "/home/jovyan/work/spark-data/raw/test_data_parquet/date=2020-04-01"
    },
    "4": {
        "init_path": "/home/jovyan/work/spark-data/raw/test_data_parquet/date=2020-05-01",
        "final_path": "/home/jovyan/work/spark-data/raw/test_data_

In [4]:
# Check schema
df.printSchema()

root
 |-- address: struct (nullable = true)
 |    |-- address: string (nullable = true)
 |    |-- address_details: struct (nullable = true)
 |    |    |-- number: string (nullable = true)
 |    |    |-- street: struct (nullable = true)
 |    |    |    |-- lat: string (nullable = true)
 |    |    |    |-- latitude: string (nullable = true)
 |    |    |    |-- long: string (nullable = true)
 |    |    |    |-- longitude: string (nullable = true)
 |    |    |    |-- street_name: string (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- country_code: string (nullable = true)
 |    |-- postal_code: string (nullable = true)
 |    |-- state: string (nullable = true)
 |-- age: string (nullable = true)
 |-- date: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- identifier: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- title: string (nullable = t

## Save data with new schema

In [5]:
data_path_dest = '/home/jovyan/work/spark-data/raw_schema_merged/test_data_{}'.format(file_format)
df.write.partitionBy('date').mode('overwrite').format(file_format).save(data_path_dest)

## Test data after schema merge

In [6]:
from pyspark.sql.functions import col, when

In [7]:
# Read merged data
df_merged = spark.read.option("mergeSchema", "true").format(file_format).load(data_path_dest)

In [8]:
# Check schema
df_merged.printSchema()

root
 |-- address: struct (nullable = true)
 |    |-- address: string (nullable = true)
 |    |-- address_details: struct (nullable = true)
 |    |    |-- number: string (nullable = true)
 |    |    |-- street: struct (nullable = true)
 |    |    |    |-- lat: string (nullable = true)
 |    |    |    |-- latitude: string (nullable = true)
 |    |    |    |-- long: string (nullable = true)
 |    |    |    |-- longitude: string (nullable = true)
 |    |    |    |-- street_name: string (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- country_code: string (nullable = true)
 |    |-- postal_code: string (nullable = true)
 |    |-- state: string (nullable = true)
 |-- age: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- identifier: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- title: string (nullable = true)
 |-- title_name: string (nullab

In [9]:
# Flatten DF
df_merged_flat = df_merged.selectExpr(flatten(df_merged.schema))

In [10]:
# Count nulls by partition and column
df_merged_flat.select(
    ["date"] + 
    [when(col(c).isNull(), 1).otherwise(0).alias(c) for c in df_merged_flat.columns if c != "date"]
).groupBy("date").sum().sort("date").toPandas()

,date,sum(address_address),sum(address_address_details_number),sum(address_address_details_street_lat),sum(address_address_details_street_latitude),sum(address_address_details_street_long),sum(address_address_details_street_longitude),sum(address_address_details_street_street_name),sum(address_city),sum(address_country),sum(address_country_code),sum(address_postal_code),sum(address_state),sum(age),sum(first_name),sum(identifier),sum(last_name),sum(occupation),sum(title),sum(title_name)
0,2020-01-01,10,10,10,10,10,10,10,10,10,10,10,10,0,0,0,0,0,10,10
1,2020-02-01,0,10,10,10,10,10,10,0,0,10,0,0,0,0,0,0,0,10,10
2,2020-03-01,0,10,10,10,10,10,10,0,0,0,0,0,0,0,0,0,0,0,10
3,2020-04-01,10,0,10,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,10
4,2020-05-01,10,0,0,10,0,10,0,0,0,0,0,0,0,0,0,0,0,10,0
5,2020-06-01,10,0,10,10,10,10,0,0,0,10,0,0,0,0,0,0,0,10,10


In [11]:
# Count by partition
df_merged_flat.select(
    col("date")
).groupBy("date").count().sort("date").toPandas()

,date,count
0,2020-01-01,10
1,2020-02-01,10
2,2020-03-01,10
3,2020-04-01,10
4,2020-05-01,10
5,2020-06-01,10
